In [8]:
import json
from bs4 import BeautifulSoup
import requests
import os

def code(soup):
    # Find all doctor info boxes
    doctor_info_boxes = soup.find_all('div', class_='doctor-info-box-in')

    doctors_data = []

    # Extract data from each doctor info box
    for box in doctor_info_boxes:
        name = soup.find('h5', class_='nomargin').text.strip()
        specialty = soup.find('div', class_='f13 doc-specialty-name').text.strip()
        fees = soup.find('div', class_='fee').find_all('div')[1].text.strip()
        degree = soup.find('div', class_='degree d-none').find_all('div')[1].text.strip()
        
        timings_divs = box.find_all('div', class_='timings-col')
        timings = [
            {
                'time': div.find('div').text.strip(),
                'days': div.find('div', class_='f12').text.strip()
            }
            for div in timings_divs
        ]   
        
        doctor_data = {
            'name': name,
            'specialty': specialty,
            'fees': fees,
            'degree': degree,
            'timings': timings,
            'hospital': 'Memo Medical School Hospital'
        }
        
        doctors_data.append(doctor_data)
    
    path = "outputs/memon-medical-hospital.json"
    # Save the extracted data to a JSON file
    if os.path.exists(path):
        with open(path, 'r') as json_file:
            existing_data = json.load(json_file)
    else:
        existing_data = []

    # Append new data to existing data
    existing_data.extend(doctors_data)

    # Save back to the JSON file
    with open(path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)


def read_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
        print(urls)
    return urls
urls=read_urls_from_file('memon-medical.txt')
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    code(soup)

['https://mmi.edu.pk/find-a-doctor/?search-type=specialty&sp=75', 'https://mmi.edu.pk/find-a-doctor/?search-type=specialty&sp=73', 'https://mmi.edu.pk/find-a-doctor/?search-type=specialty&sp=91S']


In [11]:
import json
import re

# Load the JSON data from the file
with open('outputs/memon-medical-hospital.json', 'r') as file:
    data = json.load(file)

# Function to clean text fields
def clean_text(text):
    # Remove newline characters and strip leading/trailing spaces
    text = text.replace('\n', '').strip()
    # Replace multiple spaces with a single space
    text = re.sub(' +', ' ', text)
    return text

# Iterate through each doctor entry and clean the fees and degree
for doctor in data:
    doctor['fees'] = clean_text(doctor.get('fees', ''))
    doctor['degree'] = clean_text(doctor.get('degree', ''))

# Save the cleaned data back to the file
with open('outputs/memon-medical-hospital.json', 'w') as file:
    json.dump(data, file, indent=4)


